In [3]:
!pip install QuantLib

In [4]:
import QuantLib as ql

# Step 1: Define Market Parameters
spot_price = 100  # Current stock price
strike_price = 100  # Strike price of the option
interest_rate = 0.05  # 5% annual risk-free rate
volatility = 0.2  # 20% annual volatility
time_to_maturity = 1.0  # 1 year until expiration

day_count = ql.Actual360()
calendar = ql.NullCalendar()
settlement_date = ql.Date().todaysDate()
ql.Settings.instance().evaluationDate = settlement_date

# Step 2: Construct Yield Curve and Volatility Structure
risk_free_curve = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement_date, ql.QuoteHandle(ql.SimpleQuote(interest_rate)), day_count))
volatility_structure = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(settlement_date, calendar, ql.QuoteHandle(ql.SimpleQuote(volatility)), day_count))

# Step 3: Create Black-Scholes Process
spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
bs_process = ql.BlackScholesProcess(spot_handle, risk_free_curve, volatility_structure)

# Step 4: Define European Call and Put Options
maturity_date = settlement_date + ql.Period(int(time_to_maturity * 360), ql.Days)
payoff_call = ql.PlainVanillaPayoff(ql.Option.Call, strike_price)
payoff_put = ql.PlainVanillaPayoff(ql.Option.Put, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
call_option = ql.VanillaOption(payoff_call, exercise)
put_option = ql.VanillaOption(payoff_put, exercise)

# Step 5: Set Pricing Engine (Black-Scholes Model)
bs_engine = ql.AnalyticEuropeanEngine(bs_process)
call_option.setPricingEngine(bs_engine)
put_option.setPricingEngine(bs_engine)

# Step 6: Calculate Option Prices
call_price = call_option.NPV()
put_price = put_option.NPV()
print(f"European Call Option Price: {call_price:.4f}")
print(f"European Put Option Price: {put_price:.4f}")

# Step 7: Calculate Greeks for Sensitivity Analysis
def print_greeks(option, option_type):
    print(f"\n{option_type} Option Greeks:")
    print(f"Delta: {option.delta():.4f}")
    print(f"Gamma: {option.gamma():.4f}")
    print(f"Vega: {option.vega():.4f}")
    print(f"Theta: {option.theta():.4f}")
    print(f"Rho: {option.rho():.4f}")

print_greeks(call_option, "Call")
print_greeks(put_option, "Put")


European Call Option Price: 10.4506
European Put Option Price: 5.5735

Call Option Greeks:
Delta: 0.6368
Gamma: 0.0188
Vega: 37.5240
Theta: -6.4140
Rho: 53.2325

Put Option Greeks:
Delta: -0.3632
Gamma: 0.0188
Vega: 37.5240
Theta: -1.6579
Rho: -41.8905
